In [32]:
import os, sys

base_path = os.path.join(os.getcwd(), "..")
print(f"Base Path: {base_path}")
sys.path.append(base_path)

Base Path: /Users/swkim/Documents/coding/thesis/PROMES_colab/notebook/..


In [33]:
# Load gym environment
import gym
from kube_sim_gym import *
from kube_sim_gym.envs.sim_kube_env import SimKubeEnv

In [34]:
from kube_hr_scheduler.scheduler.sim_hr_scheduler import SimHrScheduler
from kube_hr_scheduler.strategies.model.default import Model

In [35]:
import gym
import numpy as np
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.data.types import Transitions

## Expert from dataset

In [36]:
# Test version
data = np.genfromtxt('../dataset/data_expert.csv', delimiter=',', max_rows=1000000) 

# Final version
# data = np.genfromtxt('../dataset/data_expert.csv', delimiter=',') 

In [37]:
# Split the data into obs, acts, infos, next_obs, and dones
obs = data[:, :12]
acts = data[:, 12]
infos = np.empty(len(data), dtype=dict)
next_obs = data[:, 13:25]
dones = data[:, 25].astype(bool)

In [38]:
transitions = Transitions(obs, acts, infos, next_obs, dones)

In [39]:
transitions[13]

{'obs': array([0.91, 0.85, 0.73, 0.76, 0.83, 0.75, 0.82, 0.94, 0.76, 0.84, 0.05,
        0.14]),
 'acts': 1.0,
 'infos': None,
 'next_obs': array([0.86, 0.71, 0.73, 0.76, 0.83, 0.75, 0.82, 0.94, 0.76, 0.84, 0.04,
        0.06]),
 'dones': False}

## Expert from newly trained model

In [40]:
# def train_expert():
#     print("Training a expert.")

#     expert = DQN(
#         policy='MlpPolicy',
#         env=env
#     )

#     expert.learn(10000)  # Note: change this to 100000 to train a decent expert.
#     return expert

In [41]:
# def sample_expert_transitions():
#     expert = train_expert()

#     print("Sampling expert transitions.")
#     rollouts = rollout.rollout(
#         expert,
#         DummyVecEnv([lambda: RolloutInfoWrapper(env)]),
#         rollout.make_sample_until(min_timesteps=None, min_episodes=50),
#         rng=rng,
#     )
#     return rollout.flatten_trajectories(rollouts)

In [42]:
# from stable_baselines3.common.evaluation import evaluate_policy

# def eval_unit(model, env, num_eval_episodes=2):
#     rews, lens = evaluate_policy(model, env, n_eval_episodes=num_eval_episodes, return_episode_rewards=True)
#     # Takes the mean of rews elements divided by lens elements
#     mean_rew = np.mean([rew / length for rew, length in zip(rews, lens)]).round(2)

#     # Takes the difference between the maximum and minimum of reward elements
#     std_rew = (np.max(rews) - np.min(rews)) / 2

#     mean_rew = round(mean_rew, 2)
#     std_rew = round(std_rew, 2)

#     return (mean_rew, std_rew)


# def eval_set(model):
#     eval_rur_env1 = gym.make('SimKubeEnv-v0', reward_file='eval_rur.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
#     eval_rur_env2 = gym.make('SimKubeEnv-v0', reward_file='eval_rur.py', scenario_file='scenario-10l-3m-1000p-10m_unbalanced.csv')
#     eval_rur_env3 = gym.make('SimKubeEnv-v0', reward_file='eval_rur.py', scenario_file='scenario-3l-10m-1000p-10m_unbalanced.csv')

#     eval_rbd1_env1 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd1.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
#     eval_rbd1_env2 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd1.py', scenario_file='scenario-10l-3m-1000p-10m_unbalanced.csv')
#     eval_rbd1_env3 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd1.py', scenario_file='scenario-3l-10m-1000p-10m_unbalanced.csv')

#     eval_rbd2_env1 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd2.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
#     eval_rbd2_env2 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd2.py', scenario_file='scenario-10l-3m-1000p-10m_unbalanced.csv')
#     eval_rbd2_env3 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd2.py', scenario_file='scenario-3l-10m-1000p-10m_unbalanced.csv')

#     rur1 = eval_unit(model, eval_rur_env1, num_eval_episodes=2)
#     rur2 = eval_unit(model, eval_rur_env2, num_eval_episodes=2)
#     rur3 = eval_unit(model, eval_rur_env3, num_eval_episodes=2)

#     print(rur1, rur2, rur3)

#     rbd11 = eval_unit(model, eval_rbd1_env1, num_eval_episodes=2)
#     rbd12 = eval_unit(model, eval_rbd1_env2, num_eval_episodes=2)
#     rbd13 = eval_unit(model, eval_rbd1_env3, num_eval_episodes=2)

#     print(rbd11, rbd12, rbd13)

#     rbd21 = eval_unit(model, eval_rbd2_env1, num_eval_episodes=2)
#     rbd22 = eval_unit(model, eval_rbd2_env2, num_eval_episodes=2)
#     rbd23 = eval_unit(model, eval_rbd2_env3, num_eval_episodes=2)

#     print(rbd21, rbd22, rbd23)

def evalnoma(eval_model):

    eval_env0 = gym.make("SimKubeEnv-v0", reward_file='train_step_3.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
    eval_env1 = gym.make("SimKubeEnv-v0", reward_file='eval_rur.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
    eval_env2 = gym.make("SimKubeEnv-v0", reward_file='eval_rbd1.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
    eval_env3 = gym.make("SimKubeEnv-v0", reward_file='eval_rbd2.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')

    step3 = evaluate_policy(eval_model, eval_env0, n_eval_episodes=1, return_episode_rewards=True)
    rur = evaluate_policy(eval_model, eval_env1, n_eval_episodes=1, return_episode_rewards=True)
    rbd1 = evaluate_policy(eval_model, eval_env2, n_eval_episodes=1, return_episode_rewards=True)
    rbd2 = evaluate_policy(eval_model, eval_env3, n_eval_episodes=1, return_episode_rewards=True)

    print(step3, rur, rbd1, rbd2)

## Student training

In [43]:
# Initialize environment without rendering
env = gym.make("SimKubeEnv-v0", reward_file='train_step_3.py')
rng = np.random.default_rng(0)

In [44]:
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

In [45]:
evalnoma(bc_trainer.policy)

([-5877.09001268819], [8369]) ([1517.7100288439542], [8369]) ([8144.810059189796], [8369]) ([5934.309975683689], [8369])


In [46]:
# bc_trainer.train(log_interval=100000, n_batches=100000) # Test version 
bc_trainer.train(n_epochs=5, log_interval=100000) # Fianl version

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00179 |
|    entropy        | 1.79     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 88.5     |
|    loss           | 1.79     |
|    neglogp        | 1.79     |
|    prob_true_act  | 0.167    |
|    samples_so_far | 32       |
--------------------------------


99991batch [07:02, 245.13batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 100000    |
|    ent_loss       | -0.000158 |
|    entropy        | 0.158     |
|    epoch          | 3         |
|    l2_loss        | 0         |
|    l2_norm        | 1.48e+03  |
|    loss           | 0.212     |
|    neglogp        | 0.212     |
|    prob_true_act  | 0.909     |
|    samples_so_far | 3200032   |
---------------------------------


156250batch [11:01, 236.22batch/s]


In [48]:
from stable_baselines3.common.callbacks import EvalCallback
eval_env1 = gym.make("SimKubeEnv-v0", reward_file='eval_rur.py', scenario_file='scenario-5l-5m-10000p-60m_unbalanced.csv')
eval_env2 = gym.make("SimKubeEnv-v0", reward_file='eval_rbd1.py', scenario_file='scenario-5l-5m-10000p-60m_unbalanced.csv')
eval_env3 = gym.make("SimKubeEnv-v0", reward_file='eval_rbd2.py', scenario_file='scenario-5l-5m-10000p-60m_unbalanced.csv')

# Test version
# eval_callback1 = EvalCallback(eval_env1, eval_freq=100000, n_eval_episodes=3, deterministic=True, render=False)
# eval_callback2 = EvalCallback(eval_env2, eval_freq=100000, n_eval_episodes=3, deterministic=True, render=False)
# eval_callback3 = EvalCallback(eval_env3, eval_freq=100000, n_eval_episodes=3, deterministic=True, render=False)

# Final verison
eval_callback1 = EvalCallback(eval_env1, eval_freq=10000, n_eval_episodes=3, deterministic=True, render=False, log_path="results/poc_1/rur")
eval_callback2 = EvalCallback(eval_env2, eval_freq=10000, n_eval_episodes=3, deterministic=True, render=False, log_path="results/poc_1/rbd1")
eval_callback3 = EvalCallback(eval_env3, eval_freq=10000, n_eval_episodes=3, deterministic=True, render=False, log_path="results/poc_1/rbd2")

In [49]:
model1 = DQN('MlpPolicy', env, verbose=1)
model2 = DQN('MlpPolicy', env, verbose=1)
model3 = DQN('MlpPolicy', env, verbose=1)

model1.policy = bc_trainer.policy
model2.policy = bc_trainer.policy
model3.policy = bc_trainer.policy

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
evalnoma(model1)

In [ ]:
# Test version
# model1.learn(total_timesteps=5e5, 
#             log_interval=50, 
#             # progress_bar=True, 
#             callback=eval_callback1
#            )

# Final version
model1.learn(total_timesteps=1e6, 
            log_interval=50, 
            # progress_bar=True, 
            callback=eval_callback1
           )

Eval num_timesteps=10000, episode_reward=1426.30 +/- 0.00
Episode length: 1750.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 1.75e+03 |
|    mean_reward      | 1.43e+03 |
| rollout/            |          |
|    exploration_rate | 0.905    |
| time/               |          |
|    total_timesteps  | 10000    |
----------------------------------
New best mean reward!
Eval num_timesteps=20000, episode_reward=1426.30 +/- 0.00
Episode length: 1750.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 1.75e+03 |
|    mean_reward      | 1.43e+03 |
| rollout/            |          |
|    exploration_rate | 0.81     |
| time/               |          |
|    total_timesteps  | 20000    |
----------------------------------
Eval num_timesteps=30000, episode_reward=1426.30 +/- 0.00
Episode length: 1750.00 +/- 0.00
----------------------------------
| eval/               |          |
|    

KeyboardInterrupt: 

In [ ]:
# Test version
# model2.learn(total_timesteps=5e5, 
#             log_interval=50, 
#             # progress_bar=True, 
#             callback=eval_callback1
#            )

# Final version
model2.learn(total_timesteps=1e6, 
            log_interval=50, 
            # progress_bar=True, 
            callback=eval_callback2
           )

Eval num_timesteps=100000, episode_reward=1423.02 +/- 0.00
Episode length: 1609.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 1.61e+03 |
|    mean_reward      | 1.42e+03 |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 100000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0118   |
|    n_updates        | 12499    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.22e+03 |
|    ep_rew_mean      | -626     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 50       |
|    fps              | 979      |
|    time_elapsed     | 113      |
|    total_timesteps  | 110940   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.046    

In [ ]:
# Test version
# model3.learn(total_timesteps=5e5, 
#             log_interval=50, 
#             # progress_bar=True, 
#             callback=eval_callback1
#            )

# Final version
model3.learn(total_timesteps=1e6, 
            log_interval=50, 
            # progress_bar=True, 
            callback=eval_callback3
           )

Eval num_timesteps=100000, episode_reward=1423.02 +/- 0.00
Episode length: 1609.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 1.61e+03 |
|    mean_reward      | 1.42e+03 |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 100000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0443   |
|    n_updates        | 12499    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.21e+03 |
|    ep_rew_mean      | -618     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 50       |
|    fps              | 1342     |
|    time_elapsed     | 82       |
|    total_timesteps  | 110474   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0357   